# Selectors

This notebook goes over "selectors", the specifiers of inputs to feedback functions. We go over what they are, how to define them, how to use them, and how to find the appropriate selector to look up the intended data from your app or record.

## Lenses

Under the hood, selectors are "lenses", a construct to aid in accessing and modifying data structures without needing to know anything about the layout of those data structures.

We begin with the conceptual introduction to lenses.

Lenses are typically composed of a `getter` and `setter`, methods for retrieving the data named or pointed to by the lens, and for setting it.

In [24]:
from typing import Generic, TypeVar, Callable, Dict

A = TypeVar("A")
B = TypeVar("B")
C = TypeVar("C")

from dataclasses import dataclass

@dataclass
class Lens(Generic[A, B]):
    # Given a container of type A, get a value of type B from within it.
    getter: Callable[[A], B]

    # Given a container of type A and a value of type B, set the value of type B
    # within the container. Return the new container. This should not modify the
    # original.
    setter: Callable[[A, B], A]

In [22]:
# A lens for accessing and modifying "key1" in a dictionary.

key1: Lens[Dict, B] = Lens(
    getter=lambda d: d["key1"],
    setter=lambda d, v: {**d, "key1": v}
)

some_dict = dict(key1=42, key2=100)

# Use getter to get the value at "key1".
print(key1.getter(some_dict))

# Use the setter to modify/produce a new dict with a different value at "key1".
print(key1.setter(some_dict, 43))

42
{'key1': 43, 'key2': 100}


In [28]:
# Lenses can be used to devlve deeper into structures using composition. With
# two lenses, we can chain them together:
def compose(l1: Lens[A, B], l2: Lens[B, C]) -> Lens[A, C]:
    return Lens(
        getter=lambda a: l2.getter(l1.getter(a)),
        setter=lambda a, b: l1.setter(a, l2.setter(l1.getter(a), b))
    )

key1_within_key1 = compose(key1, key1)

some_nested_dict = dict(key1=dict(key1=42), key2=100)

print(key1_within_key1.getter(some_nested_dict))
print(key1_within_key1.setter(some_nested_dict, 43))

42
{'key1': {'key1': 43}, 'key2': 100}


## Lenses in trulens_eval

### Iteration

Lenses in trulens can pick out more than one item in a structure.

The getter, named `get` in trulens_eval lenses produces an iterator instead of a single item. `get_sole_item` can be used instead if it is expected that the lens picks out only one item.

### Construction

Lenses in trulens can be constructed by using the `Lens` object as if it was the object the lens is to be looking into. This produces lenses for the component named.

In [43]:
from trulens_eval.utils.serial import Lens

lens_key1 = Lens()['key1'] # lens to get value at `key1` in a dictionary.

# (We will explain `get_sole_item` vs `getter` or `get` later.)
print(lens_key1.get_sole_item(some_dict))

lens_3 = Lens()[3] # lens to get value at index 42 in a list.
print(lens_3.get_sole_item([1, 2, 3, 4, 5]))

lens_class = Lens().getter # lens to get value of attribute `getter` in an object.
print(lens_class.get_sole_item(key1)) # key1 is a lens from earlier in this notebook


42
4
<function <lambda> at 0x107129b20>


### Attributes and keys

You may have noticed that the lens `Lens()['key1']` is printed as `Lens().key1`.
This is because lenses in trulens allow for attribute lookups to operate on
dictionaries as if they were key lookups. The key-based lens constructor is thus
made equal to the attribute one. This means you use the more convenient
attribute notation to define lens for dictionary keys.

In [44]:
Lens()['key1'] == Lens().key1

True

### What is being selected from in trulens eval?

In [29]:
from trulens_eval.schema import Select

In [51]:
# The record and app are put into a dictionary for lenses to select from:
source_data = dict(
    __record__ = ...,  # the Record object is placed here
    __app__ = ... # The App object is placed here
)

# Utilities for select components.
print(Select.Record)
print(Select.App)

# User's app is located inside trulens App class under `app`.

# Calls made by app components are laid out in a record using
# `Record.layout_calls_as_app` accessible here:
print(Select.RecordCalls)

# The main call (which is added to the call list last hence the -1):
print(Select.RecordCall)

__record__
__app__
__record__.app
__record__.calls[-1]
